In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [28]:
np.__file__

'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\numpy\\__init__.py'

In [4]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import MXG, EXG

In [17]:
exa = Agilent_EXA_N9010A("SA", "EXA")
ro_awg = KeysightAWG("AWG2")
q_lo = MXG("MXG")
q_awg = KeysightAWG("AWG3")
ro_lo = EXG("EXG")

In [30]:
ro_lo = Agilent_PNA_L("PNA-L2")

In [9]:
# reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [39]:
ro_resonator_frequency = 6.57e9
if_offset =50e6

In [40]:
ro_lo.set_output_state = lambda x: x
ro_lo.set_frequency = ro_lo.set_center

In [41]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "TEST", -20)
res = cal.calibrate(lo_frequency=ro_resonator_frequency+if_offset, if_frequency=if_offset, lo_power=0,
                    ssb_power=-50, waveform_resolution=0.1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Phase:  0.4574 [ -50.065 -102.196 -116.931 ]      116.850 ]      Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -20}
Radiation parameters: {'lo_frequency': 6620000000.0, 'lo_power': 0, 'if_frequency': 50000000.0, 'ssb_power': -50, 'waveform_resolution': 0.1}
Optimization results: {'dc': -117.27996826171875, 'if': [-50.064998626708984, -102.1960220336914, -116.93069458007812]}
Optimization parameters {'dc_offsets': array([ 0.02970326, -0.13603147]), 'if_offsets': array([ 0.02815555, -0.17250821]), 'if_amplitudes': array([ 2.28732359, -2.88157622]), 'if_phase': array([ 1.43700634])}
Optimization time: 0 h 1 m 17.3 s
Finished at: 2017-04-06 18:52:00.776853


In [42]:
dm.save_IQMX_calibration(res)

In [7]:
calibrations=dm.load_IQMX_calibration_database("TEST", -20)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

## Qubit

In [18]:
qubit_frequency = 9.2e9
if_frequency = 50e6

In [26]:
cal = IQCalibrator(q_awg, exa, q_lo, "TESTQ", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=15,
                    ssb_power=-33, waveform_resolution=.1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=2000,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Phase:  0.5209 [ -43.645 -97.691 -107.485 ]       .137 ]          Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 9250000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -33, 'waveform_resolution': 0.1}
Optimization results: {'dc': -99.376426696777344, 'if': [-43.64506149291992, -97.69110107421875, -107.48456573486328]}
Optimization parameters {'dc_offsets': array([-0.08147222,  0.08185326]), 'if_offsets': array([-0.08674663,  0.0771439 ]), 'if_amplitudes': array([ 0.51958158,  0.53870046]), 'if_phase': array([ 1.63632982])}
Optimization time: 0 h 0 m 58.63 s
Finished at: 2017-04-05 14:49:43.433107


In [27]:
dm.save_IQMX_calibration(res)

In [57]:
q_lo.set_power(15)

In [75]:
calibrations = (dm.load_IQMX_calibration_database("TESTQ", -20))
cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-40, lo_frequency=qubit_frequency+if_frequency, 
                                      if_frequency=if_frequency, waveform_resolution=0.1).items()))

In [76]:
print(cal)

Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -20}
Radiation parameters: {'lo_frequency': 9250000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -40, 'waveform_resolution': 0.1}
Optimization results: {'dc': -111.25095367431641, 'if': [-39.540977478027344, -83.05058288574219, -98.06412506103516]}
Optimization parameters {'dc_offsets': array([-0.4119494 ,  0.51782169]), 'if_offsets': array([-0.48734834,  0.68734449]), 'if_amplitudes': array([ 5.17142139,  5.58796915]), 'if_phase': array([ 1.6731673])}
Optimization time: 0 h 1 m 26.33 s
Finished at: 2017-03-27 21:10:28.338740
